# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training our End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a simple - but fairly effective - logistic regression model.  More generally, however, Snorkel plugs in with many ML libraries including [TensorFlow](https://www.tensorflow.org/), making it easy to use almost any state-of-the-art model as the end extractor!

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We use the training marginals to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

In [4]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, load_as_array=True, zero_one=True)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

In [7]:
from snorkel.learning.disc_models.rnn import reRNN

train_kwargs = {
    'lr':         0.001,
    'dim':        100,
    'n_epochs':   10,
    'dropout':    0.5,
    'print_freq': 1,
    'max_sentence_length': 100
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train_cands, train_marginals, dev_candidates=dev_cands, dev_labels=L_gold_dev, **train_kwargs)

[reRNN] Training model
[reRNN] n_train=17293  #epochs=10  batch size=256
[reRNN] Epoch 0 (55.88s)	Average loss=163.240005
[reRNN] Epoch 1 (108.28s)	Average loss=160.207886
[reRNN] Epoch 2 (162.58s)	Average loss=159.574554
[reRNN] Epoch 3 (214.58s)	Average loss=159.450317
[reRNN] Epoch 4 (267.44s)	Average loss=159.404877
[reRNN] Epoch 5 (318.69s)	Average loss=159.381683
[reRNN] Epoch 6 (370.28s)	Average loss=159.370728
[reRNN] Epoch 7 (421.83s)	Average loss=159.364929
[reRNN] Epoch 8 (475.96s)	Average loss=159.343124
[reRNN] Epoch 9 (528.78s)	Average loss=159.337845
[reRNN] Training done (528.78s)


Now, we get the precision, recall, and F1 score from the discriminative model:

In [10]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.282, Recall: 0.057, F1 Score: 0.095


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [16]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test, b=.4)

Scores (Un-adjusted)
Pos. class accuracy: 0.637
Neg. class accuracy: 0.86
Precision            0.262
Recall               0.637
F1                   0.372
----------------------------------------
TP: 123 | FP: 346 | TN: 2125 | FN: 70



Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!